# Overview

This computational document looks at the growth of computing in the context of the larger picture of changing student demand, degree production, and instructional workload at UC Berkeley.  

It uses the DataScience tables framework developed for data8, but is not a tutorial document.  It does illustrate the use of tables working across several different data sets to develop insights that are not present from looking at any one of the data sets.

Run **Cell >> Run All** to execute all the cells in the notebook.  

<a id="toc">Table of Contents</a>

* [Data sources](#section_data)
* [University level trends in degrees, majors, and credits](#section_university)
 * [Degrees](#subsection_ucbdegrees)
 * [Majors](#subsection_ucbmajors)
 * [Coruses](#subsection_ucbcourses)
* [UCB productivity and workload per FTE](#UCB_perFTE)
* [Decanal level degrees, majors, and credits](#section_division)
 * [Bachelors degrees](#subsection_BA)
 * [Masters degrees](#subsection_MA)
 * [PhD degrees](#subsection_PhD)
* [Undergraduate trends normalized for overall growth](#section_undergrads)
 * [Degrees](#subsection_degrees)
 * [Majors](#subsection_majors)
 * [Course enrollments](#subsection_credits)
* [Trends within Engineering](#section_engineering)
* [Separating divisions within EECS](#section_eecs)
* [EECS degrees](#section_eecsdegrees)
* [Departmental level trends](#section_dept)
* [Fraction of degrees campus wide](#section_fraction)
* [Degree growth in related programs](#section_related)
* [STEM fields comparison](#section_stem)
* [Departmental breakout for divisions other divisions](#section_breakout)

In [ ]:
# HIDDEN
# For Tables reference see http://data8.org/datascience/tables.html
# This useful nonsense should just go at the top of your notebook.
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')
plots.rc('lines', linewidth=2, color='r')
from ipywidgets import interact
# datascience version number of last run of this notebook
version.__version__

In [ ]:
# Simple tools used in projecting tables.
def sum_cols(tbl, start):
    """Sum the columns of a table from start"""
    csum = 0
    for c in range(start, len(tbl)):
        csum += tbl[c]
    return csum

def sum_rows(tbl, start=1):
    """Sum the rows in a table beyond a start column"""
    return [sum(row[start:]) for row in tbl.rows]

def sort_cols(tbl, start=1):
    """Sort the columns of a table beyond start the values in last row."""
    labels = tbl.labels
    def col_key(label):
        return tbl.row(tbl.num_rows-1)[tbl.labels.index(label)]
    order = sorted(tbl.labels[start:], key=col_key, reverse=True)
    for label in order:
        tbl.move_to_end(label)
    return tbl
        
def top(tbl, n, start=1):
    """Create a new table containing the n largest columns."""
    tmp = tbl.copy()
    sort_cols(tmp, start)
    return tmp.select(tmp.labels[0:n+start])

def sum_collect(s):
    """Collect under sum where possible, then uniform element, then sequence """
    try:
        return sum(s)
    except TypeError:
        if np.all(np.array(s) == s[0]):
            return s[0]
        else:
            return s

def norm_by_row(tbl, base_row=0, start=1):
    """Normalize set of columns by a row"""
    labels = tbl.labels
    normed_tbl = tbl.select(tbl.labels[0:start])
    for label in labels[start:]:
        normed_tbl[label] = tbl[label]/tbl[label][base_row]
    return normed_tbl

def scale_cols(tbl, scale_factor, start=1, skip_start=False):
    """Scale set of columns"""
    labels = tbl.labels
    if skip_start:
        normed_tbl = Table()
    else:
        normed_tbl = tbl.select(tbl.labels[0:start])
    for label in labels[start:]:
        normed_tbl[label] = tbl[label]*scale_factor
    return normed_tbl

<a id="section_data"></a>


# Data

Considerable data is available through the cal-answers data warehouse.  In order to be able to manipulate it, we have extracted raw data from there on measures of productivity, Degrees awarded, which reflect the flow out of the programs, and measures of workload, Majors and student enrollment in classes, i.e., credits.  

The data model rationalizes the University structure in a revealing way.  The decanal unit is a "division".  All schools and most colleges are a single division.  Letters and Science is five divisions.  Divisions have a dean and report to the Provost.  The units within those are termed "Departments", although they may be actual departments or programs or groups.  Departments may have divisions within them, but that is not captured in the data.  To get a handle on such divisional productivity and workload requires inference from other measures.

Separate data sources describe resources.  Here we have only a 2015 FTE snapshot to work with.

As of Aug 26, 2016, the spring 2016 graduation data is not yet available, so this data is already a year old.  With the rate of growth in computing, that is important.  In 2016, the number of L&S CS grads surpassed the number of EECS grads, it became the largest major in L&S, surpassing economics, students tool more computing than all of mathematics or all of engineering, with EE included.  The data here is just prior to those crossovers, but well reflects the longer term underlying trends.

## Degree data

Degree counts are available from cal-answers since 1983.  Here they have been extracted into a csv so we can massage them to produce a departmental level record of student graduations and prorated (sum equals number of students graduating). 

In [ ]:
# Different data sets represent the name of the divisions different.  Convert to a common representation.
import re
def normalize_div(div):
    div = re.sub("-", " ", div)
    div = re.sub(" Division", "", div)
    div = re.sub(" Div", "", div)
    div = re.sub("Environmental", "Env", div)
    div = re.sub('L&S Bio Sciences','L&S Biological Sciences', div)
    div = re.sub('L&S Math & Phys Sci', 'L&S Math & Phys Sciences', div)
    div = re.sub('L&S Undergraduate', 'L&S Undergrad Studies', div)
    return div

def normalize_dept(dept):
    dept = re.sub("Comp ", "Computer ", dept)
    dept = re.sub("Rsch", "Research", dept)
    return dept

In [ ]:
raw_degrees = Table().read_table("./data/DegreeCounts-1983-2016.csv")  # Raw degree data from CalAnswers
# Only Fall 2105 degrees are represented for 2015-16 AY as of Aug 2016
degree_rows = raw_degrees.where(raw_degrees['Academic Yr'] != '2015-16')
degrees = degree_rows.drop('All Major Acad Hier Level Nm')
degrees.relabel('Academic Yr', 'Year')
degrees.relabel('Academic Department Short Nm', 'Department')
degrees.relabel('Reporting Clg School Short Nm', 'College')
degrees.relabel('Academic Division Short Nm','Division')
degrees.relabel('Student Headcount', 'Grads')
degrees.relabel('Prorated Student Major Cnt Sum', 'Prorated')
degrees['Year'] = degrees.apply(lambda x:int(x[0:4])+1, 'Year') #ending year of AY

degrees['Division'] = degrees.apply(normalize_div, 'Division')
degrees['Department'] = degrees.apply(normalize_dept, 'Department')
degrees

In [ ]:
# Crude picture of the overall degree data by department
sort_cols(degrees.where('DegLvl', 1).pivot('Department', 'Year', 'Grads', collect=sum)).plot('Year')

## Majors data

The census of majors reflects one aspect of workload that is closely related to degree production.  It integrates over the cohorts of eventual graduates moving through the program.

In [ ]:
raw_majors = Table().read_table("./data/DeptUgradCensus2005-16.csv")

majors = raw_majors.where('Semester Nm', 'Spring').select(['Academic Yr', 'Academic Division Short Nm', 
                                                           'Academic Department Short Nm','Student Headcount',
                                                          'Prorated Student Major Cnt Sum'])
majors.relabel('Academic Yr', 'Year')
majors.relabel('Academic Division Short Nm', 'Division')
majors.relabel('Academic Department Short Nm', 'Department')
majors.relabel('Student Headcount', 'Majors')
majors.relabel('Prorated Student Major Cnt Sum', 'Students')
majors['Year'] = majors.apply(lambda x:int(x[0:4])+1, 'Year')
majors['Division'] = majors.apply(normalize_div, 'Division')
majors['Department'] = majors.apply(normalize_dept, 'Department')

In [ ]:
# Small snippet of where we are
majors.where('Year', 2016).groups(['Year', 'Department'], collect=sum_collect).sort('Majors sum_collect', descending=True)

## Enrollment data

Aggregate course enrollments weighted by units is available at the subject level, for 210 subjects. Subjects are roughly at the level of departments, but some departments offer several distinct subjects, e.g., distinct languages, and some subjects are offered by more ad hoc groupings of faculty or multiple departments.

In distilling from the raw data we sum the fall, spring, and summer of each academic year. 

In [ ]:
raw_courses = Table().read_table("./data/CourseEnrollments.csv")
courses = raw_courses.select(['Academic Yr', 'Course Subject Short Nm', 'Units'])
courses.relabel('Academic Yr', 'Year')
courses.relabel('Course Subject Short Nm', 'Subject')
courses.relabel('Units', 'Credits')
courses['Year'] = courses.apply(lambda x:int(x[0:4])+1, 'Year')
courses

In [ ]:
# Crude picture of the overall data - courses by year per subject
courses_by_year = sort_cols(courses.pivot('Subject','Year', 'Credits', collect=sum))
courses_by_year.plot('Year')

In [ ]:
courses.where('Year', 2016).groups(['Year', 'Subject'], collect=sum_collect).sort('Credits sum_collect', descending=True).show()

In [ ]:
# How many subjects, i.e., fields of study, are recognized
len(np.unique(courses['Subject']))

In [ ]:
subjects = list(np.unique(courses['Subject']))
subjects

In [ ]:
# Create as close a mapping as possible from subject to administrative units
subject_map = Table.read_table("./data/subject-div.csv")
subject_map['Division'] = subject_map.apply(normalize_div, 'Div')
subject_map = subject_map.drop(['Prog', 'Div'])
subject_map['Department'] = subject_map.apply(normalize_dept, 'Dept')
subject_map = subject_map.drop('Dept')
subject_map

## Current FTE Data

FTE date at the start of the 2015-16 year.

In [ ]:
raw_fte = Table.read_table("./data/progFTE.csv")

In [ ]:
raw_fte

In [ ]:
programs = list(raw_fte['Prog'])

In [ ]:
organization = degrees.select(['Division', 'Department']).groups(['Division', 'Department']).drop('count')
organization

In [ ]:
fte = raw_fte.with_columns(['Division', raw_fte.apply(normalize_div, 'Div'),
                          'Department', raw_fte.apply(normalize_dept, 'Dept')])
fte = fte.drop(['Div', 'Dept', 'Prog'])
fte.sort('FTE', descending=True)

In [ ]:
total_fte = sum(fte['FTE'])
total_fte

In [ ]:
# Group FTE by division, i.e., decanal unit
fte_by_div = fte.group('Division', collect=sum_collect)
fte_by_div.relabel('FTE sum_collect', 'FTE')
fte_by_div.sort('FTE', descending=True).show()

In [ ]:
fte_by_dept = fte.group('Department', collect=sum_collect).sort('FTE sum_collect', descending=True)
fte_by_dept.relabel('Division sum_collect', 'Division')
fte_by_dept.relabel('FTE sum_collect', 'FTE')
fte_by_dept.show()

<a id="section_university"></a>[top](#toc)

# University Level trends

<a id="subsection_ucb_degrees"></a>

## Degrees awarded campus wide

The following graph, obtained from the degree data, shows the number of Bachelor, Masters, and PhD degrees awarded campus wide.  Advanced degree data is only available from 1990 on.  This reflects the output of these programs, rather than the effort associated with that output, as different degrees have different duration of residency.

* Today, 7647 undergrad students graduate from 123 programs with 8581 Bachelors degrees, along with 2726 Masters, and 825 PhDs.
* PhD production is fundamentally tied to size of the faculty and mentoring culture in particular fields. The rate of PhD production varies from 719 to 932.  After declines in the 90s it rose sigificantly in 2006, presumably relfecting a return to PhD studies after the Internet boom, and shows recent decline.
* Masters production grew by 30% over 25 years, but 40% since 2000.
* Bachelors degree production has grown 40% since 1990, but nearly 80% since 1985.
* Double degrees appears to have transitioned from a historical rate of ~6% to a recent rate of ~12%.  However, double majors appears to be tracking around ~7%.  This is expected to be lower than degree rate, as major declaration increases with approach to graduation.
* Total number of students and majors has increased 15% over the past decade.
* Credits delivered campus wide has increased 19% over the past decade, the increase reflecting the increasing fraction of undergraduate instruction.
* On average, an FTE produces 0.55 PhDs, 1.8 MS, and 5.7 BAs per year while carrying 17.2 Undergrads in 18.3 majors and providing 688.5 student credit hours.

In [ ]:
total_degrees = degrees.select(['Year', 'DegLvlDesc', 'Grads', 'Prorated']).groups(['Year', 'DegLvlDesc'],collect=sum)
UCB_grads_by_year = total_degrees.pivot('DegLvlDesc', 'Year', 'Grads sum', collect=sum)
print(UCB_grads_by_year.where('Year', 2015))
UCB_grads_by_year.plot('Year')
#Grads_by_year.show()
plots.title("degrees awarded @ UCB")
plots.ylabel("degrees")

In [ ]:
# Degree production per FTE
scale_cols(UCB_grads_by_year.where('Year', 2015), 1/total_fte)

In [ ]:
UCB_prorated_by_year = total_degrees.pivot('DegLvlDesc', 'Year', 'Prorated sum', collect=sum)
print(UCB_prorated_by_year.where('Year', 2015))

In [ ]:
# Relative growth from a base year
base_year = 1990
sort_cols(norm_by_row(UCB_grads_by_year.where(UCB_grads_by_year['Year']>=base_year))).plot('Year')

In [ ]:
# Number of distnct degrees
len(np.unique(degrees['Department']))

In [ ]:
# Bachelors degrees by decanal unit, i.e., Division
BAs = degrees.where('DegLvl', 1).select(['Year', 'Grads', 'Prorated'])
BAs_by_year = BAs.group('Year', collect=sum)
print(BAs_by_year.where('Year', 2015))
BAs_by_year.plot('Year')
_ = plots.title("UCB BAs Produced")

In [ ]:
BAs_by_year.with_column('Grads per Prorated', BAs_by_year['Grads sum']/BAs_by_year['Prorated sum']).select(['Year', 'Grads per Prorated']).plot('Year')

## Majors campus wide

The total number of students and majors has grown by 15% over the past 10 years.  On average 6% carry double majors, and this has remained relatively constant campus wide, although it is highly concentrated in certain disciplines - primarily those related to data science as discussed below.

In [ ]:
# Undergraduate Students and Majors
UCB_majors_by_year = majors.select(['Year', 'Majors', 'Students']).group('Year', collect=sum)
UCB_majors_by_year.plot('Year')

In [ ]:
norm_by_row(UCB_majors_by_year).plot('Year')

In [ ]:
majors_per_student_by_year = UCB_majors_by_year.select('Year').with_column('Major per student', 
                                                                             UCB_majors_by_year['Majors sum']/UCB_majors_by_year['Students sum'])
majors_per_student_by_year.show()

In [ ]:
# Total student credit hours delivered campus wide
UCB_courses_by_year = courses.drop('Subject').group('Year', collect=sum)
UCB_courses_by_year.plot('Year')

In [ ]:
norm_by_row(UCB_courses_by_year).plot('Year')

In [ ]:
# Overall picture of UCB campus wide production and workload
ucb_by_year = UCB_grads_by_year.join('Year', UCB_prorated_by_year).join('Year', UCB_majors_by_year).join('Year', UCB_courses_by_year)
ucb_by_year

<a id="UCB_perFTE"></a>

<a id="subsection_ucb_perfte"></a>[top](#toc)

## Campus-wide average production and workload per FTE

In [ ]:
# Production and workload per FTE
FTE_output = scale_cols(ucb_by_year.where('Year', 2015), 1/total_fte)
FTE_output

In [ ]:
BAs2015_per_fte = FTE_output["Bachelor"][0]
BAs2015_per_fte

In [ ]:
MAs2015_per_fte = FTE_output['Masters/Professional'][0]
MAs2015_per_fte

In [ ]:
PhDs2015_per_fte = FTE_output['Doctoral'][0]
PhDs2015_per_fte

In [ ]:
# Average number of declared majors per FTE
majors2016_per_fte = sum(majors.where(majors['Division'] != 'L&S Undeclared').where('Year', 2016)['Majors'])/total_fte
majors2016_per_fte

In [ ]:
credits2016_per_fte = FTE_output['Credits sum'][0]
credits2016_per_fte

### Mapping from course subjects to organizational entitites

Cal-Answers provides SCH per subject, from nearly 200 subjects, that align in a complicated fashion with departments and programs.  A small portion of the issue is interdisciplinary programs.  Many small departments have many distinct subjects.  Many departments have many programs and many of those have no subjects.  Painfully, we have built a mapping that is quite close. 

In [ ]:
subject_map

In [ ]:
courses_by_dept = courses.join('Subject', subject_map).drop('Prog')
courses_by_dept

<a id="section_division"></a>

## Trends in degree programs and the divisional, i.e., decanal, level

The broader trends among large majors provide an important backdrop.  
* Social Sciences experienced huge growth in the late 80's and has retained its size.  
* Humanities saw a similar 30% growth, but has decreased markedly over the recent few years.  
* L&S Undergraduate division, which is primarily interdisciplinary programs delivered by departments in other L&S divisions has grown steadily.  
* Biological Sciences saw large growth in the 90's, after the reorganization of of Biology settled in.
* Engineering has grown slowly.  
* L&S Administered programs, which contains 7 majors with other colleges or schools, has grown substantially since 2000. It is predominantly computer science.
* L&S Math and Physical Science has grown substantially since the downturn, as has the College of Natural Resources.
* Distributing the administered program degrees and majors back to their originiating divisions presents a rather different picture, especially for the College of Engineering, because CS is so large.
* Normalizing for the size of the faculty in each division, Engineering is essentially at campus average degree production, while Biological Sciences and Social Sciences BA productivity are higher, as is Social Welfare, but it is a very small program.  Humanities falls substantially below.

Collectively, we see how the last 10 years is dominated by growth in STEM, whereas Humanities and Social Sciences saw their growth in the late 80's, just as the first large boom in Computer Science ended.

In [ ]:
np.unique(degrees.where('Division', 'L&S Administered Programs')['Department'])

In [ ]:
# Bachelors degrees by decanal unit, i.e., Division
BAs_Div = degrees.where('DegLvl', 1).select(['Year','Division','Grads'])
div_BAs_by_year = BAs_Div.pivot('Division', 'Year', 'Grads', collect=sum)
sort_cols(div_BAs_by_year).where('Year',2015).barh('Year')
_ = plots.title("BAs Produced")

In [ ]:
div_BAs_by_year.plot('Year', linewidth=2)
_ = plots.title("BA production by decanal unit")

L&S Administered programs are degree programs provided by other colleges and schools, but administered through L&S because schools do not grant undergraduate degrees and because some programs are both naturally in Engineering and in a liberaal arts eduction.

In [ ]:
admin_program_degree_map = Table(['Degree', 'Division']).with_rows(
    [('L&S Chemistry', 'Clg of Chemistry'),
    ('L&S Computer Science', 'Clg of Engineering'),
    ('L&S Envir Econ & Policy', 'Clg of Natural Resources'),
    ('L&S Legal Studies', 'School of Law'),
    ('L&S Ops Research & Mgmt Sci', 'Clg of Engineering'),
    ('L&S Public Health', 'School of Public Health'),
    ('L&S Social Welfare', 'School of Social Welfare')
    ])

def map_div(division, dept):
    if division == 'L&S Administered Programs':
        for (deg, div) in admin_program_degree_map.rows:
            if deg == dept:
                return div
    return division

In [ ]:
sort_cols(degrees.where('Division', 'L&S Administered Programs').select(['Year', 'Department', 'Grads']).pivot('Department', 'Year', 'Grads', collect=sum)).plot('Year')

<a id="subsection_BA"></a>

### Bachelors degrees by decanal unit with L&S administered attributed to originating division

In [ ]:
# Revised degree data table cross divisional attribution
revised_degrees = degrees.with_column('Division', degrees.apply(map_div, ['Division', 'Department']))

With L&S administered programs attributed to their home division, we see a rather different picture of the relative undergraduate production.  College of Engineering rises above all divisions but Social Sciences, primarily due to its provision of computer science.

Importantly, the other programs, while smaller, have an increasingly computational element with the broader aspect of data science.

In [ ]:
rBAs_Div = revised_degrees.where('DegLvl', 1).select(['Year','Division','Grads'])
rdiv_BAs_by_year = rBAs_Div.pivot('Division', 'Year', 'Grads', collect=sum)
sort_cols(rdiv_BAs_by_year).where('Year',2015).barh('Year')
_ = plots.title("BAs Produced")

With this adjustment, we can see more clearly the change over the past 5 years:
* rise of Engineering undergraduate production due to computer science, and
* the corresponding rise in Math and Physical Sciences due to confluent interests in other aspects of data science, statistics and applied math.

We also see:
* the prior growth in the College of Natural Resources 2006-2012
* the growth of Biological Sciences, following the reorganization, throughout the 90's
* the growth in the late 80's of Social Science and Arts and Humanities.

The overall growth of the undergraduate program has so far allowed individual divisions to maintain degree production while other divisions experience growth.

In [ ]:
rdiv_BAs_by_year.plot('Year', linewidth=2)
_ = plots.title("BA production by decanal unit")

In [ ]:
rdiv_BAs_by_year.where(rdiv_BAs_by_year['Year'] >= 2005).plot('Year', linewidth=2)
_ = plots.title("BA production by decanal unit")

### Normalizing for size of the faculty

While the graph above shows the relative trends clearly, it leaves open the question of whether the size of the faculty across the divisions is aligned with the productivity or workload of the programs.

In [ ]:
def norm_col_by_tbl(tbl, scale_table, norm_label, norm_value, start = 1):
    """Scale set of columns by value in a scaling table. """
    labels = tbl.labels
    normed_tbl = tbl.select(tbl.labels[0:start])
    for label in labels[start:]:
        scale_row = scale_table.where(norm_label, label)[norm_value]
        if len(scale_row) == 1:
            normed_tbl[label] = tbl[label]/scale_row[0]
    return normed_tbl

In [ ]:
sort_cols(div_BAs_by_year.where('Year', 2015))

In [ ]:
# Grads per FTE
sort_cols(norm_col_by_tbl(div_BAs_by_year.where('Year', 2015), fte_by_div, 'Division', 'FTE'))

In [ ]:
# Grads after distributing administered programs to offered division
sort_cols(rdiv_BAs_by_year.where('Year', 2015))

In [ ]:
BAs2015_per_fte_by_div = sort_cols(norm_col_by_tbl(rdiv_BAs_by_year.where('Year', 2015), fte_by_div, 'Division', 'FTE'))
BAs2015_per_fte_by_div

In [ ]:
BAs2015_per_fte

In [ ]:
# Ratio of Grads per FTE to campus average
scale_cols(BAs2015_per_fte_by_div, 1/BAs2015_per_fte, 2, True)

From this normalization we can see that Engineering as a whole is essentially campus average in degree undergraduate degree production per FTE (largely due to its computer science program, as shown below), with Biology, Social Science, and Social Welfare delivering about 4/3rds the average per FTE.

<a id="subsection_MA"></a>

## Masters degrees by decanal unit, i.e., Division

At the masters level, we see profound growth amongst Business, Law, and Engineering over the past decade, with modest and mostly steady rate across the rest.

In [ ]:
MAs_Div = degrees.where('DegLvl', 2).select(['Year','Division','Grads'])
div_MAs_by_year = sort_cols(MAs_Div.pivot('Division', 'Year', 'Grads', collect=sum))
div_MAs_by_year.where('Year',2015).barh('Year')
_ = plots.title("MAs Produced by Division")

In [ ]:
div_MAs_by_year.plot('Year')
_ = plots.title("Masters production by decanal unit")

In [ ]:
# MAs production
div_MAs_by_year.where('Year',2015)

In [ ]:
# MAs per FTE
sort_cols(norm_col_by_tbl(div_MAs_by_year.where('Year', 2015), fte_by_div, 'Division', 'FTE'))

In [ ]:
# Relative to campus norm
scale_cols(sort_cols(norm_col_by_tbl(div_MAs_by_year.where('Year', 2015), fte_by_div, 'Division', 'FTE')), 1/MAs2015_per_fte)

<a id="subsection_PhD"></a>

## PhDs by Decanal unit (i.e., division)

In [ ]:
# PhDs degrees by decanal unit, i.e., Division
Ps_Div = degrees.where('DegLvl', 3).select(['Year','Division','Grads'])
div_PhDs_by_year = sort_cols(Ps_Div.pivot('Division', 'Year', 'Grads', collect=sum))
div_PhDs_by_year.where('Year', 2015).barh('Year')

In [ ]:
div_PhDs_by_year.plot('Year')
_ = plots.title("PhD production by decanal unit")

In [ ]:
div_PhDs_by_year.where('Year', 2015)

In [ ]:
sort_cols(norm_col_by_tbl(div_PhDs_by_year.where('Year', 2015), fte_by_div, 'Division', 'FTE'))

In [ ]:
scale_cols(sort_cols(norm_col_by_tbl(div_PhDs_by_year.where('Year', 2015), fte_by_div, 'Division', 'FTE')), 1/PhDs2015_per_fte)

<a id="section_undergrads"></a>

## Undergraduate trends normalized for overall growth


Given the growth in the total number of degrees, we normalize to separate shifts from overall growth.

For the past 25 years we have 
* a steady decline in social science and humanities, 
* after a rise in the 90's, biology steady
* L&S growth is primarily interdisiplinary programs and math and physical sciences
* recent growth in engineering, dominated by computer science,
* growth in math and physical sciences, dominated by statistics
* growth in CNR during 2000-2010 that has reversed since

Over the past 10 years where he have Major data, these trends are consistent and more pronounced

Normalizing for size of the faculty, Engineering (and MPS) are average campus wide while Social Sciences remains a heavier per_FTE undergraduate rate, but in both cases these figures are dominated by heavy loads in particular fields - computer science and economics, respectively.

Taking this down to course credits, the dominant trend is the growth of computer science, now exceeding Math despite near doubling of Math demands, whereas 10 years ago it was less than half.

<a id="subsection_degrees"></a>

### Degrees

Over the past 30 years, the undergraduate program has grown 70% while the faculty size and PhD program has remained roughly constant.  A better sense of student choice is obtained by normalizing for overall growth in the student population.

The growth in degree rate (and majors) is not exactly the growth in student population because the increase in junior transfers increases the size of the cohort in the upper division.

In [ ]:
UCB_grads_by_year.select(['Year', 'Bachelor']).relabel('Bachelor', 'BAs').plot('Year')

In [ ]:
UCB_grads_by_year['Bachelor'][-1]/UCB_grads_by_year['Bachelor'][0]

In [ ]:
def normalize_table(by_year):
    normed_by_year = by_year.select('Year')
    total_by_year = sum_rows(by_year)
    for label in by_year.labels[1:] :
        normed_by_year[label] = by_year[label] / total_by_year
        normed_by_year.set_format(label, PercentFormatter)
    return normed_by_year

In [ ]:
# 30 year trend in degrees as a fraction of overall campus wide production
normalize_table(rdiv_BAs_by_year).plot('Year')

In [ ]:
def forecast_table(by_year_table, past, years):
    last_year = int(by_year_table['Year'][-1])
    past_years = range(last_year+1 - past, last_year+1) # window over fit
    fore_years = range(last_year+1, last_year+1+years) # projection
    def project(lbl):
        m,b = np.polyfit(past_years, by_year_table[lbl][-past-1:-1], 1)
        return [m*year + b for year in fore_years]
    xtbl = Table().with_columns([('Year', fore_years)] + [(label, project(label)) for label in by_year_table.labels[1:]])
    return by_year_table.copy().append(xtbl)

In [ ]:
forecast_rdiv_by_year = forecast_table(rdiv_BAs_by_year, 5, 5)
forecast_rdiv_by_year.plot('Year')

In [ ]:
normalize_table(forecast_rdiv_by_year).plot('Year')

<a id="subsection_majors"></a>

### Majors

A total of 143 majors are supported across the divisions.  This metric is both workload and productivity and tends to lead `degrees` as it includes more than the graduating cohort. College differences complicate it, as Engineering students declare as freshman, whereas are `Undeclared` initially in L&S.  This is the largest segment.  

Available for a bit over a decade, the growth of L&S Administered programs is more pronounced, reflecting the growth in computer science.

Reflecting administered programs back to their colleges and schools, College of Engineering is within 80% of Social Sciences, not counting L&S Undeclared in either case.  Clearly, the non-L&S portion of COE load has lower division students attributed as majors.

In [ ]:
UCB_majors_by_year = majors.select(['Year', 'Majors']).group('Year', collect=sum_collect).relabel('Majors sum_collect', 'majors')
UCB_majors_by_year.plot('Year')

In [ ]:
division_majors = majors.select(['Year', 'Division','Majors', 'Students']).groups(['Year', 'Division'], collect=sum_collect)
print(division_majors.num_rows, " majors")
division_majors

In [ ]:
div_majors_by_year = sort_cols(division_majors.pivot('Division', 'Year', 'Majors sum_collect', collect=sum ))
div_students_by_year = sort_cols(division_majors.pivot('Division', 'Year', 'Students sum_collect', collect=sum ))

In [ ]:
# 10 year trend in majors, uncorrected for L&S Administrative program and including undeclared
div_majors_by_year.plot('Year')

In [ ]:
div_majors_by_year.where('Year', 2016).barh('Year')

In [ ]:
# 10 year trend in majors, with L&S adminstrative programs redistributed to source division
rmajors = majors.with_column('Division', majors.apply(map_div, ['Division', 'Department']))
division_rmajors = rmajors.select(['Year', 'Division','Majors', 'Students']).groups(['Year', 'Division'], collect=sum_collect)
rdiv_majors_by_year = sort_cols(division_rmajors.pivot('Division', 'Year', 'Majors sum_collect', collect=sum ))
rdiv_students_by_year = sort_cols(division_rmajors.pivot('Division', 'Year', 'Students sum_collect', collect=sum ))
rdiv_majors_by_year.where('Year', 2016).barh('Year')

In [ ]:
rdiv_majors_by_year.plot('Year')
plots.ylim(0,6000)

In [ ]:
normalize_table(rdiv_majors_by_year).plot('Year')

Forecasting this out 5 years, engineering has as many undergrads as social sciences and MPS is much larger than humanities or biological sciences

In [ ]:
forecast_table(rdiv_majors_by_year, 5, 5).plot('Year')

In [ ]:
# Majors by division w/o redistribution of administered programs
sort_cols(div_majors_by_year.where('Year', 2016))

In [ ]:
# Majors bny division
sort_cols(rdiv_majors_by_year.where('Year', 2016))

In [ ]:
# Normalized by FTE
sort_cols(norm_col_by_tbl(rdiv_majors_by_year.where('Year', 2016), fte_by_div, 'Division', 'FTE'))

In [ ]:
scale_cols(sort_cols(norm_col_by_tbl(rdiv_majors_by_year.where('Year', 2016), fte_by_div, 'Division', 'FTE')),1/majors2016_per_fte, 3, True)

<a id="subsection_credits"></a>

### Enrollment

In [ ]:
div_credits = courses.join('Subject', subject_map).select(['Division', 'Year', 'Credits']).groups(['Division', 'Year'], collect=sum)
div_credits_by_year = sort_cols(div_credits.pivot('Division', 'Year', 'Credits sum', collect=sum))
div_credits_by_year.plot('Year')

In [ ]:
normalize_table(div_credits_by_year).plot('Year')

Forecasting credits out five years, both engineering and MPS are delivering more SCH than Humanities

In [ ]:
forecast_table(div_credits_by_year, 5, 5).plot('Year')

In [ ]:
sort_cols(div_credits_by_year.where('Year', 2016))

In [ ]:
sort_cols(norm_col_by_tbl(div_credits_by_year.where('Year', 2016), fte_by_div, 'Division', 'FTE'))

In [ ]:
credits2016_per_fte

In [ ]:
# Credits per FTE relative to campus average
scale_cols(sort_cols(norm_col_by_tbl(div_credits_by_year.where('Year', 2016), 
                                     fte_by_div, 'Division', 'FTE')), 1/credits2016_per_fte, 3, True)


<a id="section_engineering"></a>

## Engineering

Engineering's undergraduate role in the university as a whole is now determined primarily by computer science.  While it remains the largest producer of PhDs and among the three large masters programs, its average standing in undergraduate prodctivity and workload is because computer science compensates for far below average rates of all the other programs.

The scale of this effect is obscure by two effects - L&S CS being attributed to the College of Letters and Science and Computer Science being a division with Electrical Engineering and Computer Sciences. 

With L&S administered programs properly attributed to engineering departments (EECS and IEOR) we find that:
* since 2014 EECS grants more bachelors degrees than all the rest of engineering
* since 2014 EECS has more majors than all the rest of engineering
* since 2013 EECS has delivered more SCH than all the rest of engineering, and in 2016 it is nearly twice that of the rest.

Forecasting out five years by each of these metrics, EECS is 2/3rds of the COE.

With separation of EECS into its two divisions, EE and CS we find:
* in 2015 CS grants more bachelors degrees than all the rest of engineering
* in 2016 CS has more majors than all the rest of engineering
* in 2016 CS has delivered more SCH than all the rest of engineering

In [ ]:
def div_grad_table_by_year(div, reflow_admin_pgms=False, degrees_tbl = degrees):
    if reflow_admin_pgms:
        degrees_tbl = degrees_tbl.with_column('Division',
                                              degrees_tbl.apply(map_div, ['Division', 'Department']))
    rows = degrees_tbl["Division"] == div
    degrees_sum = degrees.where(rows).select(['Year', 'DegLvlDesc', 'Grads', 'Prorated']).groups(['Year', 'DegLvlDesc'], collect=sum)
    grads_by_year = degrees_sum.pivot('DegLvlDesc', 'Year', 'Grads sum', collect=sum)
    return grads_by_year

In [ ]:
def combine_LS(dept):
    if dept == "L&S Computer Science":
        return "Electrical Eng & Computer Sci"
    elif dept == "L&S Ops Research & Mgmt Sci":
        return "Industrial Eng & Ops Research"
    else:
        return dept
    
def div_grads(division, degree_level, reflow_admin_pgms=False, fix_depts = False, degrees_tbl=degrees):
    degrees_tbl = degrees_tbl.copy()
    if reflow_admin_pgms:
        degrees_tbl = degrees_tbl.with_column('Division',
                                              degrees_tbl.apply(map_div, ['Division', 'Department']))
    if fix_depts:
        degrees_tbl['Department'] = degrees_tbl.apply(combine_LS, 'Department')
    div_degrees = degrees_tbl.where('Division', division)
    div_grad = div_degrees.select(['Year', 'DegLvl', 'Department', 'Grads', 'Prorated']).where('DegLvl', degree_level).drop('DegLvl')
    div_grads_by_year = sort_cols(div_grad.pivot('Department', 'Year', 'Grads', collect=sum))
    return div_grads_by_year

def div_majors(division, reflow_admin_pgms=False, fix_depts=False, majors_tbl=rmajors):
    div_majors = majors_tbl.where('Division', division)
    if reflow_admin_pgms:
        div_majors = div_majors.with_column('Department', div_majors.apply(combine_LS, 'Department'))
    if fix_depts:
        div_majors['Department'] = div_majors.apply(combine_LS, 'Department')
    div_majors_by_year = sort_cols(div_majors.pivot('Department', 'Year', 'Majors', collect=sum))
    return div_majors_by_year

def div_credits(division, courses_tbl=courses_by_dept):
    div_courses = courses_tbl.where('Division', division)
    div_courses_by_year = sort_cols(div_courses.drop(['Subject', 'Division']).pivot('Department', 'Year', 'Credits', collect=sum))
    return div_courses_by_year



In [ ]:
# College degree production w/o attribution of L&S Administered programs
sort_cols(div_grad_table_by_year('Clg of Engineering', False)).plot('Year')

In [ ]:
# College of Engineering undergrad degrees by department, w/o administered programs
eng_BAs_by_year = div_grads('Clg of Engineering', 1, False)
eng_BAs_by_year.plot('Year')

### Perspective on CoE when adminstered programs are attributed

In [ ]:
# Dispositions of degrees
div_grad_table_by_year('Clg of Engineering', True).plot('Year')

In [ ]:
div_grads('Clg of Engineering', 1, True, False).plot('Year')

In [ ]:
eng_rBAs_by_year = div_grads('Clg of Engineering', 1, True, True)
eng_rBAs_by_year.with_column("Eng-EECS", sum_rows(eng_rBAs_by_year,2)).plot('Year')

In [ ]:
eng_rBAs_by_year.where('Year', 2015)

In [ ]:
sort_cols(norm_col_by_tbl(eng_rBAs_by_year.where('Year', 2015), fte_by_dept, 'Department', 'FTE'))

In [ ]:
BAs2015_per_fte

In [ ]:
scale_cols(sort_cols(norm_col_by_tbl(eng_rBAs_by_year.where('Year', 2015), fte_by_dept, 'Department', 'FTE')),1/BAs2015_per_fte,2, True)

In [ ]:
eng_rBAs_by_year.select(['Year', 'Electrical Eng & Computer Sci']).with_column('Eng-EECS', sum_rows(eng_rBAs_by_year,2)).plot('Year')

In [ ]:
normalize_table(eng_rBAs_by_year).plot('Year')

In [ ]:
forecast_table(eng_rBAs_by_year, 5, 5).plot('Year')

In [ ]:
# Masters
eng_MSs_by_year = div_grads('Clg of Engineering', 2)
eng_MSs_by_year.plot('Year')

In [ ]:
# PhDs
eng_PhDs_by_year = div_grads('Clg of Engineering', 3)
eng_PhDs_by_year.plot('Year')

In [ ]:
# Undergraduate Engineering Majors by program
div_majors('Clg of Engineering').plot('Year')

In [ ]:
# Undergrad majors by department, including L&S adminstered programs
eng_rmajors_by_year = div_majors('Clg of Engineering', True, True)
eng_rmajors_by_year.plot('Year')

In [ ]:
normalize_table(eng_rmajors_by_year).plot('Year')

Forcasting this out, we might expect 3,000 EECS majors by 2020.

In [ ]:
forecast_table(eng_rmajors_by_year, 5, 5).plot('Year')

In [ ]:
normalize_table(forecast_table(eng_rmajors_by_year, 5, 5)).plot('Year')

Current per FTE comparison

In [ ]:
scale_cols(sort_cols(norm_col_by_tbl(eng_rmajors_by_year.where('Year', 2016), fte_by_dept, 'Department', 'FTE')),1/majors2016_per_fte,2, True)

In [ ]:
eng_rmajors_by_year

In [ ]:
eng_rmajors_by_year.select(['Year', 'Electrical Eng & Computer Sci']).with_column('Eng-EECS', sum_rows(eng_rmajors_by_year,2)).plot('Year')

In [ ]:
# Student credits
eng_courses_by_year = div_credits('Clg of Engineering')
eng_courses_by_year.plot('Year')

In [ ]:
eng_courses_by_year.select(['Year', 'Electrical Eng & Computer Sci']).with_column('Eng-EECS', sum_rows(eng_courses_by_year,2)).plot('Year')

In [ ]:
normalize_table(eng_courses_by_year).plot('Year')
forecast_table(eng_courses_by_year, 5, 5).plot('Year')

In [ ]:
credits2016_per_fte

In [ ]:
# Student credit hours per FTE relative to campus average
scale_cols(sort_cols(norm_col_by_tbl(eng_courses_by_year.where('Year', 2016), fte_by_dept, 'Department', 'FTE')),1/credits2016_per_fte,2, True)

<a id="section_eecs"></a>

## Separating divisions within EECS in college picture

In order to estimate of production rate of EE and CS within EECS, we need to utilize other data sources, either the graduation exit survey or the concentration data within the major.  From 2012 on we have a relatively simple articulation of ECE, CSE and EECS.  Prior to that we have the five options electronics, communications, computer systems, computer science and general.  The with changing demographics, the concentration data leads production changes, because it incorporates new cohorts, but it also dampens the swing because it averages over all cohorts.

As an example, the Spring 2013 exit survey shows:
* EE	17.5%
* CS	63.4%
* EECS	19.9%
* unknown	2.3%

The 12-13 concentration data shows:
* EE 28.7%
* CS 61.6%
* EECS 4.4%
* Other 5.3%

These data are useful in allowing comparison to trends at other leading institutions. Here we build a rough model, distributing the EECS pool across EE and CS on a pro rata basis in order to have a common basis for national comparisons and to gain some insight into divisional characteristics.


In [ ]:
EECS_concentrations = Table(['Year', 'CSE', 'ECE', 'EECS', 'other']).with_rows(
    [('2012', 699, 418, 183, 75),
    ('2013', 892, 415, 64, 77),
    ('2014', 950, 259, 121, 54),
    ('2015', 922, 218, 174, 54),
    ('2016', 820, 191, 293, 37)
    ]
)

In [ ]:
EECS_concentrations.plot('Year')

In [ ]:
from scipy.interpolate import UnivariateSpline
spl = UnivariateSpline([1984, 1990, 2002, 2006, 2007, 2012, 2013, 2014, 2015],
                       [0.45, 0.5, 0.65, 0.5, 0.5, 0.63, 0.68, 0.79, 0.81])

In [ ]:
# Undergraduate engineering degrees by program
adj_eng_degrees = revised_degrees.where('Division', 'Clg of Engineering').where('DegLvl', 1)
adj_eng_degrees_by_year = adj_eng_degrees.pivot('Department', 'Year', 'Grads', collect=sum)
cs_d_fraction = spl(adj_eng_degrees_by_year['Year'])
adj_eng_degrees_by_year['Computer Science'] = cs_d_fraction*adj_eng_degrees_by_year['Electrical Eng & Computer Sci'] + adj_eng_degrees_by_year['L&S Computer Science']
adj_eng_degrees_by_year['Electrical Eng'] = (1-cs_d_fraction)*adj_eng_degrees_by_year['Electrical Eng & Computer Sci']
adj_eng_degrees_by_year['Industrial Eng & Ops Research'] = adj_eng_degrees_by_year['Industrial Eng & Ops Research']+adj_eng_degrees_by_year['L&S Ops Research & Mgmt Sci']
adj_eng_degrees_by_year = sort_cols(adj_eng_degrees_by_year.drop(['Electrical Eng & Computer Sci', 'L&S Computer Science', 'L&S Ops Research & Mgmt Sci']))
adj_eng_degrees_by_year.with_column('Eng-CS', sum_rows(adj_eng_degrees_by_year, 2)).plot('Year')

In [ ]:
normalize_table(adj_eng_degrees_by_year).plot('Year')

Forecasting this out linearly is toouch because the last several years are curved so heavily upward.  Taking a linear extrapolation over a shorter past window, we can anticipate a thousand graduates per year by 2020.

In [ ]:
forecast_table(adj_eng_degrees_by_year, 3, 5).plot('Year')

In [ ]:
# Undergraduate Majors by program
adj_eng_majors = rmajors.where('Division', 'Clg of Engineering')
adj_eng_majors_by_year = adj_eng_majors.pivot('Department', 'Year', 'Majors', collect=sum)
cs_fraction = spl(adj_eng_majors_by_year['Year'])
adj_eng_majors_by_year['Computer Science'] = adj_eng_majors_by_year['Electrical Eng & Computer Sci']*cs_fraction + adj_eng_majors_by_year['L&S Computer Science']
adj_eng_majors_by_year['Electrical Eng'] = adj_eng_majors_by_year['Electrical Eng & Computer Sci']*(1-cs_fraction)
adj_eng_majors_by_year['Industrial End & Ops Research'] = adj_eng_majors_by_year['Industrial Eng & Ops Research']+adj_eng_majors_by_year['L&S Ops Research & Mgmt Sci']
adj_eng_majors_by_year = sort_cols(adj_eng_majors_by_year.drop(['Electrical Eng & Computer Sci', 'L&S Computer Science', 'L&S Ops Research & Mgmt Sci']))
# Add comparison to engineering sans CS
adj_eng_majors_by_year.with_column('Eng-CS', sum_rows(adj_eng_majors_by_year, 2)).plot('Year')

In [ ]:
forecast_table(adj_eng_majors_by_year, 3, 5).plot('Year')

In [ ]:
eng_credits = courses.join('Subject', subject_map).where('Division', 'Clg of Engineering')
def split_sub(sub, dept):
    if sub == 'Computer Science':
        return 'Computer Science'
    if sub == 'Electrical Engineering':
        return 'Electrical Eng'
    if sub == 'Electrical Eng & Comp Sci':
        return 'Electrical Eng'
    return dept
eng_credits['Department'] = eng_credits.apply(split_sub, ['Subject', 'Department'])
eng_credits = eng_credits.drop(['Dept', 'Division', 'Subject']) # .groups(['Year', 'Department'], collect=sum)
eng_credits_by_year = sort_cols(eng_credits.pivot('Department', 'Year', 'Credits', collect=sum))
eng_credits_by_year.with_column('Eng-CS', sum_rows(eng_credits_by_year, 2)).plot('Year')


Forecasting this out only five years, the situation relative to the rest of engineering is wild.

In [ ]:
forecast_table(eng_credits_by_year, 5, 5).plot('Year')

In [ ]:
EECS_fte = fte_by_dept.where('Department', 'Electrical Eng & Computer Sci')['FTE'][0]
EE_fte = 43.5
CS_fte = EECS_fte-EE_fte
adj_fte_by_dept = fte_by_dept.where(fte_by_dept['Department'] != 'Electrical Eng & Computer Sci' ).with_rows([
        ('Electrical Eng', EE_fte, 'Clg of Engineering'),
        ('Computer Science', CS_fte, 'Clg of Engineering')
    ])
adj_fte_by_dept

In [ ]:
adj_eng_degrees_by_year.where('Year', 2015)

In [ ]:
scale_cols(sort_cols(norm_col_by_tbl(adj_eng_degrees_by_year.where('Year', 2015), adj_fte_by_dept, 'Department', 'FTE')),1/BAs2015_per_fte, 2, True)

In [ ]:
adj_eng_majors_by_year.where('Year', 2016)

In [ ]:
scale_cols(sort_cols(norm_col_by_tbl(adj_eng_majors_by_year.where('Year', 2016), adj_fte_by_dept, 'Department', 'FTE')),1/majors2016_per_fte, 2, True)

In [ ]:
eng_credits_by_year.where('Year', 2016)

In [ ]:
scale_cols(sort_cols(norm_col_by_tbl(eng_credits_by_year.where('Year', 2016), adj_fte_by_dept, 'Department', 'FTE')),1/credits2016_per_fte, 2, True)

<a id="section_eecsdegrees"></a>

## Degrees from EECS

Degrees granted provides a clear sense of the flow of students, while counts of majors and credits reflect workload.

* The rate of degree production from EECS is constant for PhDs at about 70 PhDs/year.
* The masters rate has returned to its historical high of about a hundred, with the introduction of the MEng and expansion of the 5th year MS
* The EECS major has grown, but is constrained by COE admissions.  Presumably, both in the 2000's phase and in the recent one, admission was expanded to maintain some degree of quality balance with the rest of the college.
* The L&S program, which as declarations in place to try to match the EECS admissions threshold, has expanded in an unprecedented fashion.  
* In combination, the prior peak graduation rate was 470 in 2004. It dropped 34% to 312 in 2008. Climbed 2.26x to 704 in 2015-16. 

In [ ]:
def grad_table_by_year(depts):
    if type(depts) is str:
        rows = degrees["Department"] == depts
    else:
        rows = degrees["Department"] == depts[0]
        for dept in depts[1:]:
            rows = np.logical_or(rows, degrees["Department"] == dept)
    degrees_sum = degrees.where(rows).select(['Year', 'DegLvlDesc', 'Grads', 'Prorated']).groups(['Year', 'DegLvlDesc'], collect=sum)
    Grads_by_year = degrees_sum.pivot('DegLvlDesc', 'Year', 'Grads sum', collect=sum)
    return Grads_by_year

In [ ]:
EECS_grads_by_year = grad_table_by_year(['Electrical Eng & Computer Sci'])
LSCS_grads_by_year = grad_table_by_year(['L&S Computer Science'])
EECS_grads_by_year['LS CS BA'] = LSCS_grads_by_year['Bachelor']
EECS_grads_by_year.relabel('Bachelor', 'EECS BS')
EECS_grads_by_year['Bachelor'] = EECS_grads_by_year['EECS BS'] + EECS_grads_by_year['LS CS BA']
#EECS_grads_by_year.show() #uncomment to see data

In [ ]:
sort_cols(EECS_grads_by_year).plot('Year')

In [ ]:
EECS_grads_by_year.where('Doctoral').stats(ops=[min,np.mean,np.std,max])

In [ ]:
max(EECS_grads_by_year['Bachelor'])/min(EECS_grads_by_year['Bachelor'][-10:])

<a id="section_dept"></a>

# Departmental level trends


In [ ]:
def grads_by_year(degree_level=1, fix_depts = True, degrees_tbl=degrees):
    degrees_tbl = degrees_tbl.copy()
    if fix_depts:
        degrees_tbl['Department'] = degrees_tbl.apply(combine_LS, 'Department')
    dept_grads = degrees_tbl.where('DegLvl', degree_level).select(['Year', 'Department', 'Grads'])
    dept_grads_by_year = sort_cols(dept_grads.pivot('Department', 'Year', 'Grads', collect=sum))
    return dept_grads_by_year

def majors_by_year(fix_depts=True, majors_tbl=rmajors):
    if fix_depts:
        majors_tbl = majors_tbl.with_column('Department', majors_tbl.apply(combine_LS, 'Department'))
    dept_majors_by_year = sort_cols(majors_tbl.pivot('Department', 'Year', 'Majors', collect=sum))
    return dept_majors_by_year

def credits_by_year(courses_tbl=courses_by_dept):
    dept_courses_by_year = sort_cols(courses_tbl.drop(['Subject', 'Division']).pivot('Department', 'Year', 'Credits', collect=sum))
    return dept_courses_by_year

Amongst very large majors, the recent tripling of computer science is completely unlike all other developments.  Combining it with the near doubling of EECS the landscape has substantially shifted. Economics has also experienced substantial growth, suggesting that there must be substantial decreases in the many smaller programs in ths social sciences.

In [ ]:
BAs_by_year = grads_by_year()
BAs_by_year.where('Year', 2015)

In [ ]:
# Graduation rates among the 10 largest programs
top(BAs_by_year, 10).plot('Year')

Looking deeper down, we see the growth in statistics along side computer science, along with a more regular growth in math, but also the large fall off in english and sociology. 

In [ ]:
# Looking a little further down presents a rather different picture
top(BAs_by_year, 15).plot('Year')

In [ ]:
def convert_by_year_to_change(by_year_tbl, col_type, nyears, add_tbl=None):
    """Convert a table of things_by_year to one of discrete change nyears before end for each thing"""
    last = str(by_year_tbl['Year'][-1])
    prior = str(by_year_tbl['Year'][-nyears-1])
    labels = by_year_tbl.labels[1:]
    by_things_data = [(label, by_year_tbl[label][-nyears-1], by_year_tbl[label][-1]) 
                      for label in labels if by_year_tbl[label][-nyears-1] ]

    changes = Table([col_type, prior, last]).with_rows(by_things_data)
    norm_factor = sum(changes[last])/sum(changes[prior])
# Add entries that are not used in normalization
    if add_tbl:
        for label in add_tbl.labels[1:]:
            changes = changes.with_row([label, add_tbl[label][-nyears-1],add_tbl[label][-1]])
    changes['Growth'] = changes[last]-changes[prior]
    changes['Rel Growth'] = changes[last]/changes[prior]
    changes['Normed ' + prior] = changes[prior]*norm_factor
    changes['Normed Rel Growth'] = changes[last]/changes['Normed ' + prior]
    return changes.sort('Growth', descending=True)

In [ ]:
pseudo_dept_BAs_by_year = adj_eng_degrees_by_year.select(['Year', 'Computer Science', 'Electrical Eng'])

BAs_changes = convert_by_year_to_change(BAs_by_year, "BAs", 5, pseudo_dept_BAs_by_year)
BAs_changes

In [ ]:
def get_coords(tbl, key_label, key_value, x_label, y_label):
    slice = tbl.where(key_label, key_value)
    return slice[x_label][0], slice[y_label][0]

In [ ]:
dept_labels = [
    ("CS", 'Computer Science'),
    ("Stat", 'Statistics'),
    ("EECS", 'Electrical Eng & Computer Sci'),
    ("Math", 'Mathematics'),
    ("Econ", 'Economics'),
    ("CEE", 'Civil & Environmental Eng'),
    ("Hist", 'History'),
    ("Eng", 'English'),
    ("PoliSci", 'Political Science'),
    ("MCB", 'Molecular & Cell Biology'),
    ("IB", 'Integrative Biology'),
    ("EALC", 'East Asian Lang & Culture'),
    ("UGIS", 'UGIS-UG Interdisc Studies'),
    ("EE", 'Electrical Eng'),
    ("IAS Tch", 'IAS Teaching Program'),
    ("Psych", 'Psychology')
    ]

In [ ]:
BAs_changes.select(["2015", "2010"]).scatter("2010")
plots.xlim(0,800)
plots.ylim(0,800)
plots.plot([0,800],[0,800], linewidth=1)
current = "2015"
prior = "2010"
for (note, dept) in dept_labels:
    plots.annotate(note, get_coords(BAs_changes, 'BAs', dept, "2010", "2015"))

In [ ]:
BAs_changes.select(["2015", "Normed 2010"]).scatter("Normed 2010")
plots.xlim(0,800)
plots.ylim(0,800)
plots.plot([0,800],[0,800], linewidth=1)
current = "2015"
prior = "2010"
for (note, dept) in dept_labels:
    plots.annotate(note, get_coords(BAs_changes, 'BAs', dept, "Normed 2010", "2015"))

### Majors

In [ ]:
dept_majors_by_year = majors_by_year()
dept_majors_by_year.where('Year', 2016)

In [ ]:
pseudo_dept_majors_by_year = adj_eng_majors_by_year.select(['Year', 'Computer Science', 'Electrical Eng'])

majors_changes = convert_by_year_to_change(dept_majors_by_year, "majors", 5, pseudo_dept_majors_by_year)
majors_changes

In [ ]:
majors_changes.select(["2016", "2011"]).scatter("2011")
mmax = 2300
plots.xlim(0,mmax)
plots.ylim(0,mmax)
plots.plot([0,mmax],[0,mmax], linewidth=1)
for (note, dept) in dept_labels:
    if majors_changes.where('majors', dept).num_rows > 0:
        plots.annotate(note, get_coords(majors_changes, 'majors', dept, "2011", "2016"))

In [ ]:
dept_credits_by_year = credits_by_year()
dept_credits_by_year.where('Year', 2016)

In [ ]:
#pseudo_dept_credits_by_year = eng_credits_by_year.select(['Year', 'Computer Science', 'Electrical Eng'])

#credits_changes = convert_by_year_to_change(dept_credits_by_year, "credits", 5, pseudo_dept_credits_by_year)
#credits_changes

In [ ]:
#credits_changes.select(["2016", "2011"]).scatter("2011")
#mmax = 2300
#plots.xlim(0,mmax)
#plots.ylim(0,mmax)
#plots.plot([0,mmax],[0,mmax], linewidth=1)
#for (note, dept) in dept_labels:
#    if credits_changes.where('credits', dept).num_rows > 0:
#        plots.annotate(note, get_coords(credits_changes, 'credits', dept, "2011", "2016"))

<a id="section_fraction"></a>

# Fraction of degrees

It is useful to view such measures of productivity in the larger campus context. There are 1510 FTE campus wide with EECS at 90.5 FTE, so as a department it is 6% of the FTE.  In the 80's peak it reached 8.4% of the Bachellors output, but the total output was considerably smaller then.  Today it now exceeds 9%.  But the rate is increasing, so the fraction of students is considerably higher.

In [ ]:
# Fraction of FTE
EECS_fte/total_fte, CS_fte/total_fte

In [ ]:
Bach_grad = EECS_grads_by_year.select(['Year', 'Bachelor'])
Bach_grad.relabel('Bachelor', 'EECS')
Bach_grad['UCB'] = UCB_grads_by_year['Bachelor']
#Bach_grad['UCB'] = Prorated_by_year['Bachelor']
Bach_grad['Fraction EECS+CS'] = Bach_grad['EECS'] / Bach_grad['UCB']
Bach_grad.set_format('Fraction EECS+CS', PercentFormatter)
#Bach_grad.show() #uncomment to see all values

In [ ]:
Bach_grad.select(['Year', 'Fraction EECS+CS']).plot('Year')
plots.ylim(0,0.1)

In [ ]:
cs_BA_fraction = UCB_grads_by_year.select(['Year', 'Bachelor']).with_column('Computer Science', adj_eng_degrees_by_year['Computer Science'])
cs_BA_fraction['CS %']= cs_BA_fraction['Computer Science']/cs_BA_fraction['Bachelor']
cs_BA_fraction.select(['Year', 'CS %']).plot('Year')

<a id="section_related"></a>

# Degrees in related fields

In considering this broader view, it is valuable to consider our fellow travelors. 
* Statistics has long had a large instructional program, but starting from the fall off of the internet boom they have seen a profound growth of majors, which has taken off since the emergence of data science in 2010.  A very large fraction of these are double majors.  Historically there was strong double major affinity among statistics, economics, and applied math.  Now all three double with L&S CS.
* Information continues to produce a handfull of PhDs and zero Bachelors, but with the on-line MIDS have return to producing about 50 masters per year.
* Mathematics has experienced its own booms and busts, generally tracking computing, but smaller in scale.  The concentrations of pure math, applied math, and math for teachers are not broken out in the data.

These trends are reflected in graduates salary data.

In [ ]:
Stats_grads_by_year = grad_table_by_year(['Statistics'])
sort_cols(Stats_grads_by_year).plot('Year')
plots.title("Statistics Degrees")

In [ ]:
sort_cols(grad_table_by_year(['Information','Library & Info Studies'])).plot('Year')
plots.title("I School Degrees")

In [ ]:
sort_cols(grad_table_by_year(['Mathematics'])).plot('Year')
plots.title("Mathematics Degrees")

In [ ]:
grad_table_by_year(['Economics']).plot('Year')

In [ ]:
CIS_grads_by_year = grad_table_by_year(['L&S Computer Science', 'Electrical Eng & Computer Sci', 'Statistics'])
CIS_grads_by_year.plot('Year')

<a id="section_double"></a>

# Double majors

This data set gives a limited view into the prevalence od double majors.  The `Degrees` column represents
the number of degrees conferred by each unit and the `Prorated` appears to prorate the degrees earned by an
individual student over the units they receive degrees from.  So the sum of `Prorated` should be the number of actual students.  Looking at the difference in the two gives a sense of the prevalence of multiple majors.  However, there are some triple majors, so the difference slightly over estimates the number of students when looking at the student body as a whole.  

From this we can see that the number of double majors has increased from 6% to about 12%, so it is not rare.

In looking at an individual department, the number of students with double majors should exceed twice the difference, because a double major counts 1/2 to the prorated and 1 to the degrees.

* For Statistics a majority of the majors are double majors.  The fraction has 
always been high, but it shot up as the program started to grow in 2006.  As the program has grown
the double majors have settled some, but remain high and vastly different from the
student population as a whole.  With the growth, 130 of 250 graduates are double majors.
* For L&S Computer Science double major at twice the university rate and half the statistics
rates.  The fraction has grown along with the size of the program, resulting in nearly
a hundred double majors.  Many of these are doubles with statistics.  Historically, statistics,
economics and applied math common double (and triple) combinations.  Recently, all three 
double with computer science.  
* EECS double majors are relatively rare, a fraction of the campus rate.  A modest number of students graduate in joint-major programs.

In [ ]:
doubles = UCB_grads_by_year.select(['Year', 'Bachelor'])
doubles['Prorated'] =  UCB_prorated_by_year['Bachelor']
doubles['Multi'] = doubles['Bachelor'] - doubles['Prorated']
doubles['Fraction'] = doubles['Multi'] / doubles['Prorated']
doubles.set_format('Fraction', PercentFormatter)
doubles.plot('Year', 'Fraction')

In [ ]:
def doubles_by_year(depts, degrees):
    if type(depts) is str:
        rows = degrees["Department"] == depts
    else:
        rows = degrees["Department"] == depts[0]
        for dept in depts[1:]:
            rows = np.logical_or(rows, degrees["Department"] == dept)   
    BAs = degrees.where(rows).where('DegLvl', 1).select(['Year', 'Grads', 'Prorated']).group('Year',sum)
    BAs['Multi'] = 2*(BAs['Grads sum'] - BAs['Prorated sum'])
    BAs['Fraction'] = BAs['Multi'] / BAs['Grads sum']
    BAs.set_format('Fraction', PercentFormatter)
    return BAs

In [ ]:
doubles_by_year(['Statistics'], degrees).show()

In [ ]:
doubles_by_year(['Statistics'], degrees).plot('Year', 'Fraction')
doubles_by_year(['Statistics'], degrees).plot('Year', 'Multi')

In [ ]:
doubles_by_year(['L&S Computer Science'], degrees).plot('Year', 'Fraction')
doubles_by_year(['L&S Computer Science'], degrees).plot('Year', 'Multi')

In [ ]:
doubles_by_year(['Electrical Eng & Computer Sci'], degrees).plot('Year', 'Fraction')

<a id="section_stem"></a>

# STEM Fields Comparison



Current national studies examine computing in relation to other large STEM fields.  

In [ ]:
stem_BAs_by_year = BAs_by_year.select(['Year', 'Mathematics', 'Statistics', 'Physics'])
stem_BAs_by_year['Computer Science'] = pseudo_dept_BAs_by_year['Computer Science']
stem_BAs_by_year['Biology'] = BAs_by_year['Integrative Biology'] + BAs_by_year['Molecular & Cell Biology']
stem_BAs_by_year['Chemistry'] = BAs_by_year['Chemistry'] + BAs_by_year['L&S Chemistry']
edepts = ['Bioengineering',
       'Civil & Environmental Eng',
       'Engineering Science', 'Industrial Eng & Ops Research',
       'Materials Science & Eng', 'Mechanical Engineering',
       'Nuclear Engineering', 'Ocean Eng Grad Grp']
stem_BAs_by_year['Engineering'] = sum([BAs_by_year[dept] for dept in edepts]) + pseudo_dept_BAs_by_year['Electrical Eng']
sort_cols(stem_BAs_by_year).plot('Year')

<a id="section_breakout"></a>

# Breakout of departments of other divisions

In [ ]:
list(np.unique(degrees['Division']))

In [ ]:
def show_division(division):
    div_grads(division, 1, True, True).plot('Year')
    _ = plots.title(division + " BAs")
    div_majors(division, True, True).plot('Year')
    _ = plots.title(division + " majors")
    div_credits(division).plot('Year')
    _ = plots.title(division + " SCHs")
    div_grads(division, 2).plot('Year')
    _ = plots.title(division + " MAs")
    div_grads(division, 3).plot('Year')
    _ = plots.title(division + " PhDs")

In [ ]:
show_division('L&S Social Sciences')

In [ ]:
show_division('L&S Arts & Humanities')

In [ ]:
#show_division('School of Public Health')